# Indigo SP ADA Rewards Calx

**The below note book is useful to calcuate ADA rewards distributed to depositors of the Stability Pool(SP) of Indigo Protocol.
Indigo Protocol currently has 3 iAsset(iBTC, iETH, iUSD) which have their individual stability pools. These SP pools support the liquidations that occur in the protocol and the depositors of these pools are rewarded with ADA as well as INDY rewards.
The APR for these rewards is reported on a 3 months rolling is displyed on the official Indigo Protocol dashboard.**

In [8]:
import os
import pandas as pd
import time
from datetime import datetime
from typing import Final
import glob

In [9]:

REF_DATE: Final[datetime] = datetime(2022, 11, 27)
EPOCH: Final[int] = 379

def get_difference(date: datetime) -> int:
    difference = date - REF_DATE

    return difference.days

path: str = os.getcwd() + "\\SP_Staking"

if "SP_Staking_Combined" not in os.listdir(os.getcwd()):
    os.mkdir(f"{os.getcwd()}\\SP_Staking_Combined")

timestamps: list[int] = []
days = []

for filename in os.listdir(path):
    filename = filename.replace(".", "-")
    fname: list[str] = filename.split("-")
    timestamps.append(int(fname[-2]))

for timestamp in timestamps:
    full_date = time.strftime("%d %b %Y", time.gmtime(timestamp))
    date = pd.to_datetime(full_date)
    diff = get_difference(date)
    days.append(diff)

for index, file in list(enumerate(os.listdir(path))):
    df = pd.read_csv(f"{path}\\{file}")
    df.insert(0, column="timestamp", value=timestamps[index])
    df.insert(1, column="Date", value=pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(timestamps[index]))))
    df.insert(2, column="epoch", value= 379 + (days[index])//5 )
    df.to_csv(f"{os.getcwd()}\\SP_Staking_Combined\\{timestamps[index]}.csv")
#     print(file)

# Get CSV files list from a folder
path = "SP_Staking_Combined"
csv_files = glob.glob(path + "/*.csv")


# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
big_df   = pd.concat(df_list, ignore_index=True)
big_df.insert(8, column="Actual_iAsset_Staked", value=big_df['iasset_staked']/1000000)
big_df=big_df.drop("Unnamed: 0",axis=1)
columns_to_drop = ['owner']

# Drop the specified columns
big_df = big_df.drop(columns=columns_to_drop)
big_df

,timestamp,Date,epoch,asset,opened_at,iasset_staked,Actual_iAsset_Staked
0,1669585500,2022-11-27,379,iUSD,1669041696,755793662,755.793662
1,1669585500,2022-11-27,379,iUSD,1669060685,1487897614,1487.897614
2,1669585500,2022-11-27,379,iUSD,1669061163,119031809,119.031809
3,1669585500,2022-11-27,379,iUSD,1669062123,297579522,297.579522
4,1669585500,2022-11-27,379,iUSD,1669065272,162483328,162.483328
...,...,...,...,...,...,...,...
1152835,1727387100,2024-09-26,512,iBTC,1726909932,2235,0.002235
1152836,1727387100,2024-09-26,512,iBTC,1682364620,2369,0.002369
1152837,1727387100,2024-09-26,512,iBTC,1703264245,78526,0.078526
1152838,1727387100,2024-09-26,512,iBTC,1669569494,904200,0.904200


**The data for SP pool balances filtered by date is as follows:**

In [10]:

date1='2024-06-27'
filtered_df2_month = big_df.loc[(big_df['Date'] >= date1)
                     & (big_df['Date'] <= '2024-09-26')]

gk1=filtered_df2_month.groupby(['epoch','Date','asset'])
gkk1=gk1.sum()
print(gkk1.to_string())

                            timestamp      opened_at  iasset_staked  Actual_iAsset_Staked
epoch Date       asset                                                                   
494   2024-06-27 iBTC    815054707800   801294658212       36894606          3.689461e+01
                 iETH    490064539500   481949434269      402893578          4.028936e+02
                 iUSD   1330912117800  1309037392401  7344022777284          7.344023e+06
      2024-06-28 iBTC    811656439200   797900632324       36633284          3.663328e+01
                 iETH    488369552400   480273001098      402835791          4.028358e+02
                 iUSD   1330978991400  1309039295837  7350710767035          7.350711e+06
495   2024-06-29 iBTC    806538127500   792855284573       36398787          3.639879e+01
                 iETH    486674392500   478566222759      402784798          4.027848e+02
                 iUSD   1331045865000  1309054249634  7365194667311          7.365195e+06
      2024

**The oracle prices of iAssets are obtained from the Coingecko API. They are as follows:**

In [11]:
from datetime import date
import os
import pandas as pd
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

########INDY
data1=cg.get_coin_market_chart_by_id(id='indigo-dao-governance-token', vs_currency='usd', days=365)
# pd.read_json(data)
data1.pop('market_caps')
data1.pop('total_volumes')
dates1=[]
price1=[]
for i,j in data1["prices"]:
    new1=str(pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(i/1000))))
    block=new1.split(" ")[0]
    if pd.to_datetime(block)==date.today():
        break
    if block in dates1:
        break
    else:
        dates1.append(block)
    price1.append(j)


dictionary1= {'Date':dates1,'INDY Price':price1}


########BTC
data2=cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=365)
data2.pop('market_caps')
data2.pop('total_volumes')
dates2=[]
price2=[]
for i,j in data2["prices"]:
    new2=str(pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(i/1000))))
    dates2.append(new2.split(" ")[0])
    price2.append(j)

dictionary2= {'Date':dates2,'BTC Price':price2}
# dictionary2

########ETH
data3=cg.get_coin_market_chart_by_id(id='ethereum', vs_currency='usd', days=365)
data3.pop('market_caps')
data3.pop('total_volumes')
dates3=[]
price3=[]
for i,j in data3["prices"]:
    new3=str(pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(i/1000))))
    dates3.append(new3.split(" ")[0])
    price3.append(j)
dictionary3= {'Date':dates3,'ETH Price':price3}
# dictionary3

########iUSD
data4=cg.get_coin_market_chart_by_id(id='tether', vs_currency='usd', days=365)
# pd.read_json(data)
data4.pop('market_caps')
data4.pop('total_volumes')
dates4=[]
price4=[]
for i,j in data4["prices"]:
    new4=str(pd.to_datetime(time.strftime("%d %b %Y", time.gmtime(i/1000))))
    date=new4.split(" ")[0]
    if date in dates4:
        continue
    dates4.append(date)
    price4.append(j)
dictionary4= {'Date':dates4,'BTC Price':price4}

##################logic_BTC
btc_price=[]
for i in dictionary1["Date"]:
#     print(i)
    for j in set(dates2):
        if i==j:
            btc_price.append(price2[dates2.index(j)])
# len(btc_price)
# ##################logic_ETH
eth_price=[]
for i in dictionary1["Date"]:
    for j in set(dates3):
        if i==j:
            eth_price.append(price3[dates3.index(j)])
# eth_price
# ##################logic_USDT
usd_price=[]
for i in dictionary1["Date"]:
    for j in set(dates4):
        if i==j:
            usd_price.append(price4[dates4.index(j)])
# len(usd_price)


dictionary_combo= {'Date':dates1,'INDY_Price':price1, 'iBTC_Price':btc_price, 'iETH_Price':eth_price, 'iUSD_Price':usd_price}
df1 = pd.DataFrame(dictionary_combo)
print(df1.to_string())

           Date  INDY_Price    iBTC_Price   iETH_Price  iUSD_Price
0    2023-10-03    1.451502  27615.064885  1665.276425    1.000096
1    2023-10-04    1.460140  27439.121947  1656.322003    1.000235
2    2023-10-05    1.565190  27792.111277  1648.193373    1.000066
3    2023-10-06    1.595362  27435.874615  1612.994889    0.999770
4    2023-10-07    1.726257  27958.196437  1646.393402    1.000955
5    2023-10-08    1.688614  27977.543491  1635.082855    1.000754
6    2023-10-09    1.680185  27948.103652  1634.221655    1.000598
7    2023-10-10    1.696347  27593.782534  1580.177760    1.000134
8    2023-10-11    1.696380  27392.247703  1567.458226    0.999980
9    2023-10-12    1.843751  26842.190439  1564.916203    0.999544
10   2023-10-13    1.769496  26729.137206  1536.812805    0.999501
11   2023-10-14    1.742202  26841.136221  1550.966873    0.999721
12   2023-10-15    1.747204  26863.183569  1555.456208    1.000193
13   2023-10-16    1.699651  27150.297001  1557.760809    0.99

**ADA discussed before the ADA obtained through liquidations is distributed to SP depositors on a pro rata basis. The total value gained(in $) by the pools for each of the iAssets are as follows:**

In [12]:
import urllib.request
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
from typing import Final

# open a connection to a URL using urllib, read the data from the url and decode byte obj to string.
data  = urllib.request.urlopen('https://analytics.indigoprotocol.io/api/liquidations').read().decode("utf-8")

df_ada_sp = pd.DataFrame(json.loads(data))
df_ada_sp = df_ada_sp.drop(["slot", "output_hash", "output_index", "updated_at"], axis=1)

df_ada_sp["created_at"] = pd.Series(df_ada_sp["created_at"], dtype="string")

df_ada_sp[["date", "time"]] = df_ada_sp.created_at.apply(lambda x: pd.Series(str(x).split("T")))

df_ada_sp = df_ada_sp.drop(["time", "created_at"], axis=1)
df_ada_sp["date"] = pd.to_datetime(df_ada_sp["date"])

df_ada_sp["ref_date"] = pd.to_datetime("2022-11-27")

# df_ada_sp

def calc_epoch(x,y) -> int:
    return 379 + ((x - y).days)//5

vec_epoch_calc = np.vectorize(calc_epoch)
df_ada_sp["epoch"] = vec_epoch_calc(df_ada_sp["date"].to_list(), df_ada_sp["ref_date"].to_list())

df_ada_sp[["oracle_price", "ada_price"]] = df_ada_sp[["oracle_price", "ada_price"]].astype(float)

# print(df_ada_sp.dtypes)

def get_value_gained(a,b,c,d) -> float:
    return ((a/1000000)-((b/1000000)*c))*d

vec_value_gained = np.vectorize(get_value_gained)
df_ada_sp["value_gained"] = vec_value_gained(df_ada_sp["collateral_absorbed"].to_list(), df_ada_sp["iasset_burned"].to_list(), df_ada_sp["oracle_price"].to_list(), df_ada_sp["ada_price"].to_list())

######change
columns_to_drop = ['ref_date']

# Drop the specified columns
df_ada_sp_dropped = df_ada_sp.drop(columns=columns_to_drop)
df_ada_sp_dropped

new2= df_ada_sp_dropped.loc[(df_ada_sp['date'] >= date1) & (df_ada_sp['date'] <= '2024-09-26')]
rew_buffer3=new2.groupby(['date', 'asset']).sum()
rew_buffer4=rew_buffer3.groupby(['asset'])
print("==========================================Rewards============================================")
print(rew_buffer4.sum().to_string())
print("=============================================================================================")

==========================================Rewards============================================
         id  collateral_absorbed  iasset_burned  oracle_price  ada_price  epoch  value_gained
asset                                                                                        
iETH   2279           2359220248          19643   8380.294887   0.420009    499    921.754320
iUSD   9111          41718058759       38214784     10.623860   1.531539   1990  17396.083705


**Now, the mean value of deposits(in $) in the SP pools during the above specified duration is:**

In [13]:
#iBTC
temp_arr_btc=[]
temp_arr_eth=[]
temp_arr_usd=[]
new1=filtered_df2_month
buffer=new1.groupby(['Date','asset'])
# buffer12=buffer11.sum()
# print(buffer12.to_string())

ada_reward_array11=new1.Date.unique()
ada_reward_array11.sort()
# ada_reward_array11

for i in ada_reward_array11:
    buffer11= df1[df1['Date'] == i]
    temp11=buffer11.iBTC_Price.tolist()[0]
    
    
    buffer12=buffer.get_group((i,'iBTC')).sum()
    buffer13=buffer12.Actual_iAsset_Staked
    
    value1=temp11*buffer13
    temp_arr_btc.append(value1)
mean_btc=sum(temp_arr_btc)/len(temp_arr_btc) 
print("==========================================SP Staking============================================")
print("iBTC: "+str(mean_btc))

##comment block while calculating for ibtc/iusd
for j in ada_reward_array11:
    buffer21= df1[df1['Date'] == j]
    temp21=buffer21.iETH_Price.tolist()[0]
    

    buffer22=buffer.get_group((j,'iETH')).sum()
    buffer23=buffer22.Actual_iAsset_Staked
    
    value2=temp21*buffer23
    temp_arr_eth.append(value2)
mean_eth=sum(temp_arr_eth)/len(temp_arr_eth)
print("iETH: "+str(mean_eth))

for k in ada_reward_array11:
    buffer31= df1[df1['Date'] == k]
    temp31=buffer31.iUSD_Price.tolist()[0]
        
    
    buffer32=buffer.get_group((k,'iUSD')).sum()
    buffer33=buffer32.Actual_iAsset_Staked
    
    value3=temp31*buffer33
    temp_arr_usd.append(value3)
mean_usd=sum(temp_arr_usd)/len(temp_arr_usd)
print("iUSD: "+str(mean_usd))
print("================================================================================================")

==========================================SP Staking============================================
iBTC: 1858769.1368098108
iETH: 720464.4494072668
iUSD: 4899857.293059694


**The APR for the iAssets can be obtained easily from the above data:**

iBTC=(0/1858769.1368098108) * (4 * 100)% =0%

iETH=(921.754320/720464.4494072668) * (4 * 100)% = 0.51%

iUSD=(17396.083705/4899857.293059694) * (4 * 100)% = 1.42%